In [1]:
import os
import random

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from evaluator import evaluate
from data_loader import load_kdd_cup_urc, load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4, load_power_demand # Univariate Datasets
from data_loader import load_nasa, load_ecg, load_gesture, load_smd # Multivariate Datasets

from tqdm.notebook import tqdm
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, GRU, Reshape, Dropout, GaussianNoise, Concatenate, Lambda, RepeatVector, Bidirectional

# THESE LINES ARE FOR REPRODUCIBILITY
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
def Skip_AE(seq_length, dim, N_RES=3, N_LAYERS=2, BATCH_SIZE=128):
    tf.keras.backend.clear_session()
    
    en_inputs = []
    shared_latents = []
    for n in range(N_RES):
        selected_t = [t for t in range(0, seq_length, 2**n)]
        en_input = Input(shape=[len(selected_t), dim])
        en_inputs.append(en_input)

    de_outputs = []
    for n in range(N_RES):
        for l in range(N_LAYERS):
            X = en_inputs[n] if l == 0 else X
            X = GaussianNoise(0.5)(X) if l == 0 else X
            X = Bidirectional(GRU(64, return_sequences=False if l + 1 == N_LAYERS else True))(X) # add dropout?
        h = Dense(32, activation='relu', kernel_regularizer='l1')(X)
        shared_latents.append(h)

    for n in range(N_RES):
        selected_t = [t for t in range(0, seq_length, 2**n)]
        for l in range(N_LAYERS):
            if l == 0:
                X = Concatenate()(shared_latents)
                X = Dense(128, activation='relu')(X)
                X = Concatenate()([X, shared_latents[n]])
                X = RepeatVector(len(selected_t))(X)
            X = Bidirectional(GRU(64, return_sequences=False if l + 1 == N_LAYERS else True))(X)
        rec_x = Dense(len(selected_t) * dim)(X)
        rec_x = Reshape([len(selected_t), dim])(rec_x)
        de_outputs.append(rec_x)

    model = Model(inputs=en_inputs, outputs=de_outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")

    return model

### Yahoo S5

In [4]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [5]:
for loader in [load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4]:
    datasets = loader(64, 1)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        N_RES = 3

        seq_length, dim = X_train.shape[1], X_train.shape[2]
        X_train_reverse = np.flip(X_train, axis=1)
        X_test_reverse = np.flip(X_test, axis=1)

        X_train_by_res = []
        X_test_by_res = []
        X_train_by_res_reverse = []
        
        for n in range(N_RES):
            selected_t = [t for t in range(0, seq_length, 2**n)]
            X_train_by_res.append(X_train[:, selected_t, :])
            X_test_by_res.append(X_test[:, selected_t, :])
            X_train_by_res_reverse.append(X_train_reverse[:, selected_t, :])
        
        model = Skip_AE(seq_length, dim, N_RES, N_LAYERS=2, BATCH_SIZE=128)
        history = model.fit(X_train_by_res, X_train_by_res_reverse, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
                            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
        
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test_by_res)]
        scores = evaluate(X_test, X_test_rec[0], y_tests[i], is_reconstructed=True)
    
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

yahoo_A1 0.999999948484851 0.9848484765973171 0.9999999981470107
yahoo_A1 0.9999999416666693 0.9166666376815789 0.9999999913849765
yahoo_A1 0.9999999456521764 0.9130434632832964 0.9999999953614762
yahoo_A1 0.999999930000003 0.5999999693333345 0.9999999797237571
yahoo_A1 0.9230768591716004 0.7196935725032695 0.9394841129580422
yahoo_A1 0.9230768591716004 0.7176059162845598 0.8916666536808864
yahoo_A1 0.999999948484851 0.9848484764310015 0.9999999981601732
yahoo_A1 0.9999999492063519 0.9920634873045555 0.9999999987844084
yahoo_A1 0.9999999492063519 0.9920634871921966 0.9999999987844085
yahoo_A1 0.9999999487179513 0.8974358928532621 0.9999999983719279
yahoo_A1 0.9962824771410039 0.9925110843432494 0.9999510511927483
yahoo_A1 0.9999999487654349 0.9753086357533925 0.9999999984095602
yahoo_A1 0.9999999483870992 0.967741927972047 0.9999999980537634
yahoo_A1 0.999999944444447 0.9444444226876735 0.9999999941537467
yahoo_A1 0.9999999487013013 0.9870129799850018 0.9999999983645983
yahoo_A1 0.9999

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/92 [00:00<?, ?it/s]

yahoo_A3 0.25396823169564314 0.01041666744313249 -0.0
yahoo_A3 0.442477841491114 0.2007682077068614 0.3258348398203997
yahoo_A3 0.999999949275365 0.9927536188423729 0.9999999989442364
yahoo_A3 0.25396823169564314 0.0026455034406369553 -0.0
yahoo_A3 0.9950494544591242 0.9945121121224598 0.9994803674008619
yahoo_A3 0.41726615387920135 0.30359647587951855 0.5559760092961191
yahoo_A3 0.5852089616298454 0.251850025654074 0.12291966403039958
yahoo_A3 0.45614031550631234 0.0863235492647818 0.22332155724185798
yahoo_A3 0.5565216946956556 0.5818209242502106 0.7125913565587488
yahoo_A3 0.7134502462945892 0.5670579303029345 0.46513119814004256
yahoo_A3 0.637770854195864 0.5771960591224656 0.5520191867874078
yahoo_A3 0.5783521396394755 0.5799018505807767 0.5339337616770949
yahoo_A3 0.24933684806900963 0.08980547313542196 0.4361160728066099
yahoo_A3 0.6766916843462069 0.5500754970282786 0.31467027828751615
yahoo_A3 0.11563168070466742 0.0012077301643646464 -0.0
yahoo_A3 0.9783783278729025 0.9917629

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

yahoo_A4 0.9688580808467357 0.9893134628228955 0.9983158982915654
yahoo_A4 0.2694736607805006 0.009065629824517188 0.08387158034586203
yahoo_A4 0.41726615387920135 0.12567035322404257 0.26240593844745963
yahoo_A4 0.3076922815088779 0.06409727934041237 0.22532469142678216
yahoo_A4 0.7412140102195622 0.8279016004230586 0.8056366982465082
yahoo_A4 0.7927272242988459 0.832950494062082 0.8772449310971777
yahoo_A4 0.9666666158923637 0.9869240510207291 0.9979482433405215
yahoo_A4 0.8815165370319652 0.9367697797791019 0.9840190751119157
yahoo_A4 0.5942491593197877 0.3045360683616841 0.38626853673190853
yahoo_A4 0.5232067114102114 0.5649103430611196 0.6489383005670752
yahoo_A4 0.637770854195864 0.4066215793652738 0.5059943661315502
yahoo_A4 0.439862505412079 0.21164851991908262 0.558544216416582
yahoo_A4 0.45070419027970904 0.0015974454049713467 -0.0
yahoo_A4 0.3551401575753364 0.14418167407900936 0.432115259830743
yahoo_A4 0.07439824225732532 0.001179246171092864 -0.0
yahoo_A4 0.81333328332444

In [6]:
yahoo_results = pd.DataFrame(total_scores)
yahoo_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
yahoo_A1,0.900721,0.818349,0.899435
yahoo_A2,0.872528,0.560448,0.961456
yahoo_A3,0.683438,0.574377,0.631628
yahoo_A4,0.612167,0.474574,0.560439


### NASA

In [7]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [8]:
for loader in [load_nasa]:
    datasets = loader(100, 100)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]

        N_RES = 3

        seq_length, dim = X_train.shape[1], X_train.shape[2]
        X_train_reverse = np.flip(X_train, axis=1)
        X_test_reverse = np.flip(X_test, axis=1)

        X_train_by_res = []
        X_test_by_res = []
        X_train_by_res_reverse = []
        
        for n in range(N_RES):
            selected_t = [t for t in range(0, seq_length, 2**n)]
            X_train_by_res.append(X_train[:, selected_t, :])
            X_test_by_res.append(X_test[:, selected_t, :])
            X_train_by_res_reverse.append(X_train_reverse[:, selected_t, :])
        
        model = Skip_AE(seq_length, dim, N_RES, N_LAYERS=2, BATCH_SIZE=128)
        history = model.fit(X_train_by_res, X_train_by_res_reverse, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
                            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
        
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test_by_res)]
        scores = evaluate(X_test, X_test_rec[0], y_tests[i], is_reconstructed=True)
        
        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

D1 0.25300017462469454 0.1354973447388353 0.3957106874447126
D2 0.2699530282282525 0.580331526100397 0.513043477732262


In [9]:
nasa_results = pd.DataFrame(total_scores)
nasa_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.253000,0.135497,0.395711
D2,0.269953,0.580332,0.513043


### SMD

In [10]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [11]:
for loader in [load_smd]:
    datasets = loader(64, 1)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]

        N_RES = 3

        seq_length, dim = X_train.shape[1], X_train.shape[2]
        X_train_reverse = np.flip(X_train, axis=1)
        X_test_reverse = np.flip(X_test, axis=1)

        X_train_by_res = []
        X_test_by_res = []
        X_train_by_res_reverse = []
        
        for n in range(N_RES):
            selected_t = [t for t in range(0, seq_length, 2**n)]
            X_train_by_res.append(X_train[:, selected_t, :])
            X_test_by_res.append(X_test[:, selected_t, :])
            X_train_by_res_reverse.append(X_train_reverse[:, selected_t, :])
        
        model = Skip_AE(seq_length, dim, N_RES, N_LAYERS=2, BATCH_SIZE=128)
        history = model.fit(X_train_by_res, X_train_by_res_reverse, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
                            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
        
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test_by_res)]
        scores = evaluate(X_test, X_test_rec[0], y_tests[i], is_reconstructed=True)
      
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

smd 0.20232181144029696 0.5556857798256027 0.521888680406877
smd 0.590886280944706 0.47389795223936687 0.8937595589594608
smd 0.16979264501627309 0.13722790427744214 0.5554861163111511
smd 0.22018344041404736 0.23317577062930836 0.5732138542761814
smd 0.689655126355723 0.7135990132801731 0.9046644080959915
smd 0.7443028055443416 0.8395272888160692 0.9103655804502616
smd 0.45931030628237607 0.4505349749340864 0.7132689921791673
smd 0.40011017361624973 0.3956330841858173 0.789538435214881
smd 0.5697229685017566 0.6000682207643923 0.8721205268603118
smd 0.2596273980668859 0.3063727598643879 0.5431190497734129
smd 0.7002966860467945 0.6958113354590681 0.8258015862838908
smd 0.2969778069506946 0.311836250455808 0.671219797335445
smd 0.7556917187949171 0.7548720560567956 0.9370280282840387
smd 0.5251491512954548 0.5148442848441896 0.7110624636389428
smd 0.8066297842470146 0.8426384119966593 0.9564497481883735
smd 0.9677418850899386 0.6730995648291123 0.9809677285777375
smd 0.7643821429023572

In [12]:
smd_results = pd.DataFrame(total_scores)
smd_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
smd,0.548197,0.526993,0.786997


### ECG

In [13]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [14]:
for loader in [load_ecg]:
    datasets = loader(32, 16)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        N_RES = 3

        seq_length, dim = X_train.shape[1], X_train.shape[2]
        X_train_reverse = np.flip(X_train, axis=1)
        X_test_reverse = np.flip(X_test, axis=1)

        X_train_by_res = []
        X_test_by_res = []
        X_train_by_res_reverse = []
        
        for n in range(N_RES):
            selected_t = [t for t in range(0, seq_length, 2**n)]
            X_train_by_res.append(X_train[:, selected_t, :])
            X_test_by_res.append(X_test[:, selected_t, :])
            X_train_by_res_reverse.append(X_train_reverse[:, selected_t, :])
        
        model = Skip_AE(seq_length, dim, N_RES, N_LAYERS=2, BATCH_SIZE=128)
        history = model.fit(X_train_by_res, X_train_by_res_reverse, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
                            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
        
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test_by_res)]
        scores = evaluate(X_test, X_test_rec[0], y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

D1 0.642857092346942 0.521180694932693 0.6050347182306737
D2 0.3999999493333391 0.28191957982385296 0.6487825946058183
D3 0.2352940692041611 0.08421957400232341 0.5252191917853902
D4 0.4444443901234624 0.29754916918207863 0.7306497089576638
D5 0.4999999487244943 0.29917783951159516 0.8523309933289677
D6 0.26666661577778683 0.11107774780176724 0.5069791276695598
D7 0.10344823888229587 0.02700085153025625 0.5914366594972826
D8 0.19354836940686937 0.08288577893704768 0.41451596933771245
D9 0.3909774115665127 0.18626792798183406 0.34865724320430674


In [15]:
ecg_results = pd.DataFrame(total_scores)
ecg_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.642857,0.521181,0.605035
D2,0.400000,0.281920,0.648783
D3,0.235294,0.084220,0.525219
D4,0.444444,0.297549,0.730650
D5,0.500000,0.299178,0.852331
D6,0.266667,0.111078,0.506979
D7,0.103448,0.027001,0.591437
D8,0.193548,0.082886,0.414516
D9,0.390977,0.186268,0.348657


### Power Demand

In [16]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [17]:
for loader in [load_power_demand]:
    datasets = loader(64, 1)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        N_RES = 3

        seq_length, dim = X_train.shape[1], X_train.shape[2]
        X_train_reverse = np.flip(X_train, axis=1)
        X_test_reverse = np.flip(X_test, axis=1)

        X_train_by_res = []
        X_test_by_res = []
        X_train_by_res_reverse = []
        
        for n in range(N_RES):
            selected_t = [t for t in range(0, seq_length, 2**n)]
            X_train_by_res.append(X_train[:, selected_t, :])
            X_test_by_res.append(X_test[:, selected_t, :])
            X_train_by_res_reverse.append(X_train_reverse[:, selected_t, :])
        
        model = Skip_AE(seq_length, dim, N_RES, N_LAYERS=2, BATCH_SIZE=128)
        history = model.fit(X_train_by_res, X_train_by_res_reverse, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
                            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
        
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test_by_res)]
        scores = evaluate(X_test, X_test_rec[0], y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc']) 

  0%|          | 0/1 [00:00<?, ?it/s]

power_demand 0.3606206231571944 0.15239158356120688 0.36340129434921037


In [18]:
power_results = pd.DataFrame(total_scores)
power_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
power_demand,0.360621,0.152392,0.363401


### 2D Gesture

In [19]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [20]:
for loader in [load_gesture]:
    datasets = loader(64, 1)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        N_RES = 3

        seq_length, dim = X_train.shape[1], X_train.shape[2]
        X_train_reverse = np.flip(X_train, axis=1)
        X_test_reverse = np.flip(X_test, axis=1)

        X_train_by_res = []
        X_test_by_res = []
        X_train_by_res_reverse = []
        
        for n in range(N_RES):
            selected_t = [t for t in range(0, seq_length, 2**n)]
            X_train_by_res.append(X_train[:, selected_t, :])
            X_test_by_res.append(X_test[:, selected_t, :])
            X_train_by_res_reverse.append(X_train_reverse[:, selected_t, :])
        
        model = Skip_AE(seq_length, dim, N_RES, N_LAYERS=2, BATCH_SIZE=128)
        history = model.fit(X_train_by_res, X_train_by_res_reverse, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
                            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
        
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test_by_res)]
        scores = evaluate(X_test, X_test_rec[0], y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

  0%|          | 0/1 [00:00<?, ?it/s]

gesture 0.6085219207333007 0.4602819203456862 0.7621153082235194


In [21]:
gesture_results = pd.DataFrame(total_scores)
gesture_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
gesture,0.608522,0.460282,0.762115
